In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/emilypickett/Documents/InstaCart Basket Project'

In [21]:
#import order_merge
df_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_grouped.pkl'))

In [4]:
#1 agg mean of order_number
df_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


the results differ slightly to the subset some departments such as 21 and 10 changed more department 10 changing from 18.6 to 20.9 and department 21 changing from 25.5 to 22.9. Where other departments only had a slight change of .1. This might be due to the departments with less of a change had more data in the first million rows (subset) verse the departments that changed more significantly had more data after the subset.

In [22]:
#5 Do prices for each loyalty_flag differ on price
loyalty_stats = df_merge.groupby('loyalty_flag')['prices'].describe()

In [13]:
print(loyalty_stats)

                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal customer    10284093.0  10.386336  328.017787  1.0  4.2  7.4  11.2   
New customer       6243990.0  13.294670  597.560299  1.0  4.2  7.4  11.3   
Regular customer  15876776.0  12.495717  539.720919  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal customer    99999.0  
New customer      99999.0  
Regular customer  99999.0  


In [23]:
#6 create spender flag based on avg price by user_id
df_merge['avg_price'] = df_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/zk/lxc3sp7n5rqf4k_sl172k5xh0000gn/T/ipykernel_82734/3311408029.py:2: FutureWarning: The provided callable <function mean at 0x118b35620> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_merge['avg_price'] = df_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [27]:
df_merge.loc[df_merge['avg_price'] < 10, 'spender'] = 'Low spender'

In [28]:
df_merge.loc[df_merge['avg_price'] > 10, 'spender'] = 'High spender'

In [29]:
df_merge[['avg_price', 'spender']].head(10)

,avg_price,spender
0,6.935811,Low spender
1,6.935811,Low spender
2,7.930208,Low spender
3,4.972414,Low spender
4,4.972414,Low spender
5,6.935398,Low spender
6,5.957576,Low spender
7,6.680000,Low spender
8,7.162500,Low spender
9,7.162500,Low spender


In [32]:
#7 create frequency flag basied on median days_since_prior_order
df_merge['median_prior_order'] = df_merge.groupby(['user_id'])['prices'].transform(np.median)

/var/folders/zk/lxc3sp7n5rqf4k_sl172k5xh0000gn/T/ipykernel_82734/4072466340.py:2: FutureWarning: The provided callable <function median at 0x119967880> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_merge['median_prior_order'] = df_merge.groupby(['user_id'])['prices'].transform(np.median)


In [33]:
df_merge.loc[df_merge['median_prior_order'] > 20, 'frequency'] = 'Non-frequent_customer'

/var/folders/zk/lxc3sp7n5rqf4k_sl172k5xh0000gn/T/ipykernel_82734/3958318904.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent_customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merge.loc[df_merge['median_prior_order'] > 20, 'frequency'] = 'Non-frequent_customer'


In [34]:
df_merge.loc[(df_merge['median_prior_order'] <= 20) & (df_merge['median_prior_order'] > 10), 'frequency'] = 'Regular customer'

In [35]:
df_merge.loc[df_merge['median_prior_order'] <= 10, 'frequency'] = 'Frequent_customer'

In [37]:
df_merge[['median_prior_order', 'frequency']].head(10)

,median_prior_order,frequency
0,6.6,Frequent_customer
1,6.6,Frequent_customer
2,7.5,Frequent_customer
3,4.8,Frequent_customer
4,4.8,Frequent_customer
5,7.8,Frequent_customer
6,5.8,Frequent_customer
7,5.6,Frequent_customer
8,7.2,Frequent_customer
9,7.2,Frequent_customer


In [38]:
#export dataframe as pickle
df_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_grouped_2.pkl'))